In [6]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc
import imageio
import os
import pandas as pd
import mahotas as mh

In [7]:
# folder path
dir_path = r'BestSegmentations/'
# list to store files
res = [] # files name
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [12]:
end2_iou_otsu = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'], index=range(370))
for i in range(0, 370, 1):
    print(i)
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    end2_otsu = cv2.imread(('GrayHeatmapsEnd2end/' + res1.split(".")[0] + '_CAM4.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(end2_otsu) # == np.shape(orig)
    otsu_threshold, image_otsu = cv2.threshold(end2_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    intersection = np.logical_and(image_otsu, segm1)
    union = np.logical_or(image_otsu, segm1)
    iou = np.sum(intersection) / np.sum(union)
    end2_iou_otsu.loc[i] = (res1.split(".")[0], otsu_threshold, iou)
    # Save image with contours--------------------------------------------------------------------------------------------
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
    contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_otsu, hierarchy1 = cv2.findContours(image_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
    cv2.drawContours(orig, contours_otsu, -1, (255,255,0), 6)
    cv2.imwrite(('ImageOtsu/End2end1/' + res1.split(".")[0] + '_CAM4OT.png'), orig)
    # plt.imshow(orig,cmap='gray')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
end2_iou_otsu

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,2,0
1,173ca_256_R_MLO,0,0.0224279
2,168ca_248_L_MLO,0,0.0107651
3,137ca_208_R_CC,21,0.112169
4,133ca_203_L_MLO,1,0
...,...,...,...
365,14ca_19_R_MLO,2,0
366,140ca_211_L_MLO,1,0.0401321
367,23ca_38_L_CC,8,0.0254889
368,13ca_18_R_CC,3,0


In [14]:
np.sum(end2_iou_otsu['IoU'])/370

0.02183141134059624

In [8]:
end2_iou = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    end2 = cv2.imread(('GrayHeatmapsEnd2end/' + res1.split(".")[0] + '_CAM4.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(end2)
    iou_score = []
    for n in range(0, 255, 1):
        th , image = cv2.threshold(end2, n, 255, cv2.THRESH_BINARY) 
        intersection = np.logical_and(image, segm1)
        union = np.logical_or(image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        iou_score.append(iou)
    threshold = iou_score.index(np.max(iou_score))
    IoU = np.max(iou_score)
    end2_iou.loc[i] = (res1.split(".")[0], threshold, IoU)
    # Save image with contours--------------------------------------------------------------------------------------------
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
    th , image = cv2.threshold(end2, threshold, 255, cv2.THRESH_BINARY)
    contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_sweep, hierarchy1 = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
    cv2.drawContours(orig, contours_sweep, -1, (255,255,0), 6)
    cv2.imwrite(('ImageSweep/End2end/' + res1.split(".")[0] + '_CAM4SW.png'), orig)

In [4]:
end2_iou

NameError: name 'end2_iou' is not defined

In [5]:
np.sum(end2_iou['IoU'])/370

NameError: name 'end2_iou' is not defined

In [4]:
end2_iou_otsu_ = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'])
end2_iou_no_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
otsu_ = 0
no_ = 0
for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    end2_otsu = cv2.imread(('GrayHeatmapsEnd2end/' + res1.split(".")[0] + '_CAM4.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(end2_otsu)
    otsu_threshold, image_otsu = cv2.threshold(end2_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    inter_out = np.sum(np.logical_and(image_otsu, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_otsu)
        new_image = np.zeros(np.shape(image_otsu))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        end2_iou_otsu_.loc[otsu_] = (res1.split(".")[0], otsu_threshold, iou)
        otsu_ += 1
        #plt.imshow(image_otsu)
        #show()
        #plt.imshow(new_image)
        #show()
        #plt.imshow(segm1)
        #show()
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        #cv2.imwrite(('ImageOtsu/End2end/' + res1.split(".")[0] + '_CAM4OT.png'), orig)
    else:
        end2_iou_no_.loc[no_] = (res1.split(".")[0], otsu_threshold)
        no_ += 1

In [27]:
end2_iou_no_

,image_index,threshold_otsu
0,123ca_187_L_CC,2.0
1,133ca_203_L_MLO,1.0
2,151ca_225_L_MLO,1.0
3,179ca_264_L_MLO,1.0
4,134ca_204_R_MLO,2.0
...,...,...
206,108ca_166_R_MLO,3.0
207,10ca_14_R_CC,0.0
208,57ca_88_L_CC,1.0
209,14ca_19_R_MLO,2.0


In [30]:
end2_iou_otsu_
print('IoU End2end Otsu = ', np.sum(end2_iou_otsu_['IoU'])/otsu_)

IoU End2end Otsu =  0.055232908509117926


In [5]:
otsu_

159

In [33]:
end2_iou_sw_ = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
end2_iou_n_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
sw_ = 0
n_ = 0

for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    end2_sw = cv2.imread(('GrayHeatmapsEnd2end/' + res1.split(".")[0] + '_CAM4.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(end2_sw)
    iou_score = []
    for n in range(0, 255, 1):
        th , image = cv2.threshold(end2_sw, n, 255, cv2.THRESH_BINARY)      
        intersection = np.logical_and(image, segm1)
        union = np.logical_or(image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        iou_score.append(iou)
    threshold = iou_score.index(np.max(iou_score))
    IoU = np.max(iou_score)
    th , image_sw = cv2.threshold(end2_sw, threshold, 255, cv2.THRESH_BINARY) 
    inter_out = np.sum(np.logical_and(image_sw, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_sw)
        new_image = np.zeros(np.shape(image_sw))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        end2_iou_sw_.loc[sw_] = (res1.split(".")[0], th, iou)
        sw_ += 1
        #plt.imshow(image_otsu)
        #show()
        #plt.imshow(new_image)
        #show()
        #plt.imshow(segm1)
        #show()
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageSweep/End2end/' + res1.split(".")[0] + '_CAM4SW.png'), orig)
    else:
        end2_iou_n_.loc[n_] = (res1.split(".")[0], otsu_threshold)
        n_ += 1

In [36]:
end2_iou_sw_
print('IoU End2end Sweep = ', np.sum(end2_iou_sw_['IoU'])/sw_)

IoU End2end Sweep =  0.09192331899954231


In [34]:
end2_iou_sw_

,image_index,threshold,IoU
0,123ca_187_L_CC,0.0,0.052066
1,173ca_256_R_MLO,0.0,0.199694
2,168ca_248_L_MLO,0.0,0.010765
3,137ca_208_R_CC,37.0,0.164440
4,133ca_203_L_MLO,0.0,0.002968
...,...,...,...
234,14ca_19_R_MLO,0.0,0.005208
235,140ca_211_L_MLO,1.0,0.065476
236,23ca_38_L_CC,12.0,0.033600
237,13ca_18_R_CC,0.0,0.004062


In [35]:
end2_iou_n_

,image_index,threshold_otsu
0,151ca_225_L_MLO,10.0
1,179ca_264_L_MLO,10.0
2,134ca_204_R_MLO,10.0
3,97ca_152_R_CC,10.0
4,121ca_185_R_CC,10.0
...,...,...
126,33ca_55_L_MLO,10.0
127,16ca_23_R_CC,10.0
128,108ca_166_R_MLO,10.0
129,10ca_14_R_CC,10.0
